In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import time
import os

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transforms['train'])
val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

class_names = train_dataset.classes


model_transfer = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

from IPython.display import Image
Image("https://neurohive.io/wp-content/uploads/2018/11/vgg16-1-e1542731207177.png")


# for param in model_transfer.features.parameters():
#     #print(param.shape)
#     param.requires_grad = False

model_transfer.classifier[6] = nn.Linear(model_transfer.classifier[6].in_features, len(class_names))


model_transfer = model_transfer.to(DEVICE)
optimizer  = optim.Adam(model_transfer.classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for param in model_transfer.features.parameters():
    print(param.requires_grad)


for param in model_transfer.classifier.parameters():
    print(param.requires_grad)



Using PyTorch version: 2.6.0+cu124  Device: cuda


100%|██████████| 170M/170M [00:02<00:00, 83.1MB/s] 
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 191MB/s]  


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [5]:
from torchinfo import summary
summary(model_transfer)

Layer (type:depth-idx)                   Param #
VGG                                      --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       1,792
│    └─ReLU: 2-2                         --
│    └─Conv2d: 2-3                       36,928
│    └─ReLU: 2-4                         --
│    └─MaxPool2d: 2-5                    --
│    └─Conv2d: 2-6                       73,856
│    └─ReLU: 2-7                         --
│    └─Conv2d: 2-8                       147,584
│    └─ReLU: 2-9                         --
│    └─MaxPool2d: 2-10                   --
│    └─Conv2d: 2-11                      295,168
│    └─ReLU: 2-12                        --
│    └─Conv2d: 2-13                      590,080
│    └─ReLU: 2-14                        --
│    └─Conv2d: 2-15                      590,080
│    └─ReLU: 2-16                        --
│    └─MaxPool2d: 2-17                   --
│    └─Conv2d: 2-18                      1,180,160
│    └─ReLU: 2-19                

In [ ]:
BATCH_SIZE = 32

def train(model, train_loader, optimizer, log_interval):
    global output, label
    model.train()
    correct = 0
    train_loss = 0
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()


        _, prediction = torch.max(output, dim=1 )
        # 정답을 맞춘 갯수
        # acc += sum(idx == label)
        correct += prediction.eq(label.view_as(prediction)).sum().item()


        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image),
                len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                loss.item()))
           
    train_loss /= (len(train_loader.dataset) / BATCH_SIZE)
    train_accuracy = 100. * correct / len(train_loader.dataset)


    return train_loss, train_accuracy



def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0


    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()


    test_loss /= (len(test_loader.dataset) / BATCH_SIZE)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

EPOCHS = 5
loss_hist_train     = [0] * EPOCHS
accuracy_hist_train = [0] * EPOCHS
loss_hist_valid     = [0] * EPOCHS
accuracy_hist_valid = [0] * EPOCHS

for epoch in range(1, EPOCHS + 1):
    loss_, acc_ = train(model_transfer, train_loader, optimizer, log_interval = 200)
    loss_hist_train[epoch] = loss_
    accuracy_hist_train[epoch] = acc_
    test_loss, test_accuracy = evaluate(model_transfer, val_loader)
    loss_hist_valid[epoch] = test_loss
    accuracy_hist_valid[epoch] = test_accuracy
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))



Train Epoch: 1 [0/50000 (0%)]	Train Loss: 2.501790
Train Epoch: 1 [6400/50000 (13%)]	Train Loss: 0.905717
Train Epoch: 1 [12800/50000 (26%)]	Train Loss: 0.791066
Train Epoch: 1 [19200/50000 (38%)]	Train Loss: 0.677942
Train Epoch: 1 [25600/50000 (51%)]	Train Loss: 1.077359
Train Epoch: 1 [32000/50000 (64%)]	Train Loss: 1.194925
Train Epoch: 1 [38400/50000 (77%)]	Train Loss: 0.555558
Train Epoch: 1 [44800/50000 (90%)]	Train Loss: 0.722624

[EPOCH: 1], 	Test Loss: 0.5032, 	Test Accuracy: 84.44 % 

Train Epoch: 2 [0/50000 (0%)]	Train Loss: 0.522824
